# Import packages and API Setup

## Import packages

In [1]:
# Data manipulation
import datetime as dt
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None) # Remove column display limit
import pandas_datareader.data as web

# Stats
import statsmodels.api as sm
from statsmodels.regression.rolling import RollingOLS

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Data Fetch
import requests
import yfinance as yf


## API setup and configuration 

### Alpha Vantage

[Alpha Vantage API Documentation](https://www.alphavantage.co/documentation/#)

In [2]:
# Import dotenv_values and load the API key from .env file
from dotenv import dotenv_values 
secrets = dotenv_values("secrets/.env")
alpha_key = secrets["alphavantage_key"]

#### Search Endpoint

[Documentation](https://www.alphavantage.co/documentation/#symbolsearch)  

The Search Endpoint returns the best-matching symbols and market information based on keywords of your choice. The search results also contain match scores that provide you with the full flexibility to develop your own search and filtering logic.  

In [3]:
# Function to search for symbols based on a keyword
def av_search(keyword):
    # Base URL
    base_url = "https://www.alphavantage.co/query?function=SYMBOL_SEARCH&keywords={keyword}&apikey={apikey}"

    # API key for authentication
    api_key = alpha_key

    # Construct the full URL
    url = base_url.format(keyword=keyword, apikey=api_key)

    # Send HTTP GET request to fetch data
    response = requests.get(url)
    
    # Parse the response as JSON
    response_json = response.json()
    
    # Create a DataFrame from the 'bestMatches' section of the JSON response
    df_results = pd.DataFrame(response_json.get('bestMatches', []))

    return df_results

#### Time Series Daily

[Documentation](https://www.alphavantage.co/documentation/#daily)  

This API returns raw (as-traded) daily time series (date, daily open, daily high, daily low, daily close, daily volume) of the global equity specified, covering 20+ years of historical data. The OHLCV data is sometimes called "candles" in finance literature. 

In [4]:
# Function to fetch daily stock data
def av_daily(symbol, output_size):
    # Base URL
    base_url = "https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&outputsize={output_size}&apikey={apikey}"

    # API key for authentication
    api_key = alpha_key

    # Construct the full URL
    url = base_url.format(symbol=symbol, output_size=output_size, apikey=api_key)

    # Send HTTP GET request to fetch data
    response = requests.get(url)

    # Parse the response as JSON
    data = response.json()

    # Extract the daily time series data, with a default to an empty dictionary
    time_series = data.get("Time Series (Daily)", {})

    # Create a DataFrame from the time series data
    ts_df = pd.DataFrame.from_dict(time_series, orient='index')

    # Convert the index to datetime for easier manipulation and analysis
    ts_df.index = pd.to_datetime(ts_df.index)

    # Return the resulting DataFrame
    return ts_df

#### Market News & Sentiment

[Documentation](https://www.alphavantage.co/documentation/#news-sentiment)

This API returns live and historical market news & sentiment data from a large & growing selection of premier news outlets around the world, covering stocks, cryptocurrencies, forex, and a wide range of topics such as fiscal policy, mergers & acquisitions, IPOs, etc.


The stock/crypto/forex symbols of your choice. For example: tickers=IBM will filter for articles that mention the IBM ticker; tickers=COIN,CRYPTO:BTC,FOREX:USD will filter for articles that simultaneously mention Coinbase (COIN), Bitcoin (CRYPTO:BTC), and US Dollar (FOREX:USD) in their content.

In [5]:
# Function to fetch news sentiment data
def av_news(tickers):
    # Base URL 
    base_url = "https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers={tickers}&apikey={apikey}"

    # API key for authentication
    api_key = alpha_key

    # Construct the full URL
    url = base_url.format(tickers=tickers, apikey=api_key)

    # Send HTTP GET request to fetch data
    response = requests.get(url)

    # Parse the response as JSON
    data = response.json()

    # Normalize the 'feed' data into a DataFrame
    feed_df = pd.json_normalize(data['feed'])

    # Return the DataFrame with the news sentiment data
    return feed_df

# Building unsupervised learning trading strategy

#

## ETL

### Standard and Poor's 500 (S&P 500/SPY) list as of 04/25/2024

In [9]:
# Load dataset from CSV
spy_list = pd.read_csv("data/SPY_holdings_042524.csv")

spy_list.head()

,#,Company,Symbol,Portfolio%
0,1,Microsoft Corp,MSFT,7.11%
1,2,Apple Inc.,AAPL,5.69%
2,3,Nvidia Corp,NVDA,4.78%
3,4,Amazon.com Inc,AMZN,3.85%
4,5,"Meta Platforms, Inc. Class A",META,2.56%


In [37]:
# # Replace dots with dashes for yfinance query
# spy_list['Symbol'] = spy_list['Symbol'].str.replace('.', '-')

# # Extract symbols only
# symbols_list = spy_list['Symbol'].unique().tolist()

# # Set start and end dates
# end_date = '2024-04-24'
# start_date = pd.to_datetime(end_date) - pd.DateOffset(365*10)

# # Download yfinance data
# df = yf.download(tickers=symbols_list,
#                  start=start_date,
#                  end=end_date
#                  ).stack()

# # Rename the index levels
# df.index.names = ['date', 'ticker']

# # Change columns names to lowercase
# df.columns = df.columns.str.lower()

In [32]:
# # Save to csv
# df.to_csv("data/yfdata.csv")

In [40]:
# Load saved data
df = pd.read_csv("data/yfdata.csv")
df.head()

,date,ticker,adj close,close,high,low,open,volume
0,2014-04-28,A,35.332298,38.433475,39.291843,37.811157,39.234619,4206582.0
1,2014-04-28,AAL,33.146801,35.160000,36.720001,34.340000,36.410000,13149300.0
2,2014-04-28,AAPL,18.688702,21.217501,21.276787,20.448214,20.457144,669485600.0
3,2014-04-28,ABBV,33.703716,50.869999,50.919998,49.500000,49.549999,7932200.0
4,2014-04-28,ABT,31.691114,38.369999,38.419998,37.779999,38.279999,4433600.0


In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1231853 entries, 0 to 1231852
Data columns (total 8 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   date       1231853 non-null  object 
 1   ticker     1231853 non-null  object 
 2   adj close  1231853 non-null  float64
 3   close      1231853 non-null  float64
 4   high       1231853 non-null  float64
 5   low        1231853 non-null  float64
 6   open       1231853 non-null  float64
 7   volume     1231853 non-null  float64
dtypes: float64(6), object(2)
memory usage: 75.2+ MB
